In [1]:
from decouple import Config, RepositoryEnv
import psycopg2 as pg
import pandas as pd


In [2]:
config = Config(RepositoryEnv('./.env'))
HOST = config.get('REMOTE_HOST')
UNAME = config.get('UNAME')
PW = config.get('PASSWORD')
DB_NAME = config.get('DB_NAME')
BASE_PATH = config.get('BASE_PATH')

In [3]:
# get post counts per platform
with pg.connect("host='{}' port={} dbname='{}' user={} password={}".format(HOST, 5432, DB_NAME, UNAME, PW)) as conn:
    sql_query = """SELECT 
    c.language,
    c.platform,
    c.date,
    l.V2_GR_bin,
    l.V2_NWO_bin,
    COUNT(*) AS count
FROM 
    content c
JOIN 
    labels_consp l ON c.label_consp = l.id
GROUP BY 
    c.language, 
    c.platform, 
    c.date, 
    l.V2_GR_bin, 
    l.V2_NWO_bin
ORDER BY 
    c.language, 
    c.platform, 
    c.date, 
    l.V2_GR_bin, 
    l.V2_NWO_bin;
"""
    dat = pd.read_sql_query(sql_query, conn)

C:\Users\kiliab90\AppData\Local\Temp\ipykernel_15872\374357235.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dat = pd.read_sql_query(sql_query, conn)


In [4]:
dat

,language,platform,date,v2_gr_bin,v2_nwo_bin,count
0,eng,alt_news,2010-06-29,False,False,1
1,eng,alt_news,2011-01-01,False,False,1
2,eng,alt_news,2011-01-03,False,False,1
3,eng,alt_news,2011-01-03,False,True,1
4,eng,alt_news,2011-01-04,False,False,2
...,...,...,...,...,...,...
93410,ger,twitter,2021-12-30,False,True,11
93411,ger,twitter,2021-12-30,True,False,4
93412,ger,twitter,2021-12-31,False,False,16
93413,ger,twitter,2021-12-31,False,True,2


In [5]:
from src.query_neovex import get_query_wrapper

In [6]:
# EXAMPLE 1 : basic query
# get all alternative and legacy news posts for august
## OPTION 1: specify get_query_wrapper function
NEOVEXwrapper11 = get_query_wrapper(platform=['alt_news', 'legacy_news'], daterange=('2020-08-01','2020-08-31'))
dat11 = NEOVEXwrapper11.execute_query()

## OPTION 2: specify using 'set_' methods
NEOVEXwrapper12 = get_query_wrapper()
NEOVEXwrapper12.set_platform(['alt_news', 'legacy_news'])
NEOVEXwrapper12.set_daterange('2020-08-01','2020-08-31')
dat12 = NEOVEXwrapper12.execute_query()

In [7]:
# EXAMPLE 2: aggregate query
# EXAMPLE 2.1 : get post counts per platform
NEOVEXwrapper2 = get_query_wrapper()
dat21 = NEOVEXwrapper2.sum_rows(group_by="platform")
# EXAMPLE 2.2 : get post counts per platform and date
dat22 = NEOVEXwrapper2.sum_rows(group_by=["platform","date"])

In [9]:
dat12

,id,date,timestamp,text,text_prep,title,platform,subplatform,language,content_id,label_liwc,label_consp,created_at,updated_at
0,477,2020-08-19,None,"Speaking in the border city of Yuma, Ariz., Pr...","Speaking in the border city of Yuma, Ariz., Pr...","Speaking on Immigration, Trump Revives The Spe...",legacy_news,NYT,eng,477,NaN,37220,2024-08-13 18:02:13.287056,2024-08-13 18:02:13.287056
1,478,2020-08-17,None,A major European show of work made during coro...,A major European show of work made during coro...,"They're in Lockdown, and It Shows",legacy_news,NYT,eng,478,NaN,37221,2024-08-13 18:02:13.287056,2024-08-13 18:02:13.287056
2,479,2020-08-08,None,Scottish polling shows a majority favor indepe...,Scottish polling shows a majority favor indepe...,"After Quitting E.U., London Moves to Thwart Sc...",legacy_news,NYT,eng,479,NaN,37222,2024-08-13 18:02:13.287056,2024-08-13 18:02:13.287056
3,480,2020-08-08,None,But history has a way of confounding those who...,But history has a way of confounding those who...,Trump and Allies Think They Know Who Counts,legacy_news,NYT,eng,480,NaN,37223,2024-08-13 18:02:13.287056,2024-08-13 18:02:13.287056
4,481,2020-08-01,None,"If Joe Biden wins the November election, Brita...","If Joe Biden wins the November election, Brita...","After Years of Mollifying Trump, U.K. Ponders ...",legacy_news,NYT,eng,481,NaN,37224,2024-08-13 18:02:13.287056,2024-08-13 18:02:13.287056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237,15947631,2020-08-02,None,Heavyweight mustachioed actor Wilford Brimley ...,Heavyweight mustachioed actor Wilford Brimley ...,Actor Wilford Brimley Dies At 85 Mustachioed a...,alt_news,infowars,eng,51844,NaN,20263580,2024-09-17 13:15:13.821524,2024-09-17 13:15:13.821524
1238,15948607,2020-08-13,None,France is getting more deeply involved against...,France is getting more deeply involved against...,"France Deploys Jets, Warship to East Mediterra...",alt_news,infowars,eng,52820,NaN,20264556,2024-09-17 13:15:13.821524,2024-09-17 13:15:13.821524
1239,15948608,2020-08-13,None,President Trump just announced a “historic” de...,President Trump just announced a “historic” de...,Trump’s “Historic” Israel-UAE Peace Plan to No...,alt_news,infowars,eng,52821,NaN,20264557,2024-09-17 13:15:13.821524,2024-09-17 13:15:13.821524
1240,15948609,2020-08-05,None,Iranian state media has announced that a US Na...,Iranian state media has announced that a US Na...,US Navy Seizes Iran-Bound Ship Carrying Pharma...,alt_news,infowars,eng,52822,NaN,20264558,2024-09-17 13:15:13.821524,2024-09-17 13:15:13.821524


In [3]:
dat3

,platform,date,count
0,alt_news,2010-06-29,1
1,alt_news,2011-01-01,1
2,alt_news,2011-01-03,2
3,alt_news,2011-01-04,2
4,alt_news,2011-01-05,2
...,...,...,...
19208,twitter,2021-12-28,589
19209,twitter,2021-12-29,638
19210,twitter,2021-12-30,677
19211,twitter,2021-12-31,597


In [ ]:
# label (inclusion/exclusion), platform, subplatform, string match, language, zeitraum, author
# get dataframe, sum rows, sum per time unit

In [ ]:
NEOVEXwrapper = get_query_wrapper()

In [ ]:
NEOVEXwrapper.set_platform('alt_news')
NEOVEXwrapper.set_language('eng')
NEOVEXwrapper.set_daterange('2020-08-01','2020-08-31')
NEOVEXwrapper.get_criteria()
# results = db_wrapper.execute_query()
# print(results)
# total_rows = db_wrapper.sum_rows()
# print(total_rows)
# monthly_aggregation = db_wrapper.sum_per_time_unit('MONTH')
# print(monthly_aggregation)
# db_wrapper.close()


In [ ]:
NEOVEXwrapper.execute_query()

# Kilians queries

In [68]:
with pg.connect("host='{}' port={} dbname='{}' user={} password={}".format(HOST, 5432, DB_NAME, UNAME, PW)) as conn:
    sql_query = """SELECT 
    c.subplatform,
    c.platform,
    c.language,
    COUNT(*) FILTER (WHERE l.V1_bin = 'true') AS true_count,
    COUNT(*) FILTER (WHERE l.V1_bin = 'false') AS false_count,
    COUNT(*) AS total_count,
    COUNT(*) FILTER (WHERE l.V1_bin = 'true')::FLOAT / COUNT(*) AS true_share
FROM 
    content c
JOIN 
    labels_consp l ON c.label_consp = l.id
WHERE
    c.language = 'eng'
    AND (c.platform = 'alt_news' OR c.platform = 'legacy_news')
GROUP BY 
    c.language, 
    c.platform, 
    c.subplatform
ORDER BY 
    c.platform, 
    c.subplatform,
    c.language
"""
dat = pd.read_sql_query(sql_query, conn)

In [69]:
dat

,subplatform,platform,language,true_count,false_count,total_count,true_share
0,blaze,alt_news,eng,674,671,1345,0.501115
1,breitbart,alt_news,eng,5412,10053,15465,0.349952
2,dailycaller,alt_news,eng,2722,2808,5530,0.492224
3,gatewaypundit,alt_news,eng,3836,593,4429,0.866110
4,infowars,alt_news,eng,6602,1838,8440,0.782227
5,occupyDems,alt_news,eng,474,157,631,0.751189
6,tpm,alt_news,eng,510,393,903,0.564784
7,NYT,legacy_news,eng,1283,15135,16418,0.078146
8,USAtoday,legacy_news,eng,225,1625,1850,0.121622
9,WP,legacy_news,eng,1310,10389,11699,0.111975


In [53]:
with pg.connect("host='{}' port={} dbname='{}' user={} password={}".format(HOST, 5432, DB_NAME, UNAME, PW)) as conn:
    sql_query = """SELECT 
    c.text_prep,
    c.title,
    c.language,
    c.platform,
    c.subplatform,
    c.date,
    l.V1_bin,
    l.V2_GR_bin,
    l.V2_NWO_bin
FROM 
    content c
JOIN 
    labels_consp l ON c.label_consp = l.id
WHERE
    c.language = 'eng'
    AND (c.platform = 'alt_news' OR c.platform = 'legacy_news')
ORDER BY
    c.date
"""
dat = pd.read_sql_query(sql_query, conn)

In [54]:
dat

,text_prep,title,language,platform,subplatform,date,v1_bin,v2_gr_bin,v2_nwo_bin
0,"I've had $100,000 burning in my pocket for the...","Reward: $100,000 for Full 'JournoList' Archive...",eng,alt_news,breitbart,2010-06-29,True,False,False
1,"QANDIL, Iraq\nHIGH in the craggy mountains of ...",A Kurdish Rebel Softens His Tone for Skeptical...,eng,legacy_news,NYT,2011-01-01,False,True,False
2,"A dozen or so years ago, I\r\n had a horrifica...","Fright from the '50s,on sturdy modern paper",eng,legacy_news,WP,2011-01-01,False,False,False
3,"Juanita White may be 75 \r\nyears old, but on ...",Ward 8 ties hopes for revival to Gray,eng,legacy_news,WP,2011-01-01,False,False,False
4,I am trying to sell my \r\ncondominium unit an...,To tell or not to tell about the bedbugs nearby,eng,legacy_news,WP,2011-01-01,False,False,False
...,...,...,...,...,...,...,...,...,...
73066,Tucker Carlson has beaten me again – every tim...,HISS STORY: Tucker Carlson goes off the deep-end,eng,alt_news,occupyDems,2023-01-21,True,False,False
73067,Plans to vote on a strict Republican anti-immi...,“TONE DEAF”: Republican Immigration bill dead ...,eng,alt_news,occupyDems,2023-01-22,False,True,False
73068,I spend far too much time thinking about what ...,"SINFUL: Republicans are coming for your porn, ...",eng,alt_news,occupyDems,2023-02-17,True,False,False
73069,The billionaire benefactor who’s been supplyin...,REVEALED: The SHOCKING art collection of the b...,eng,alt_news,occupyDems,2023-04-08,False,False,False
